In [2]:
import os
import requests
import json
import pandas as pd
from datetime import datetime
from dotenv import load_dotenv
import mysql.connector
from mysql.connector import Error
import pyodbc

In [3]:
def check_rate_limits():
    """
    Fetch and display the API rate limits from the API-Football service.
    This helps monitor the number of requests available before hitting the limit(100 calls per day).
    """

    # Load environment variables from the 'keys.env' file saved in my directory
    load_dotenv('keys.env')
    # Retrieve the API key from environment variables
    RAPIDAPI_KEY = os.getenv('RAPIDAPI_KEY')
    # Check if the API key is loaded correctly
    if not RAPIDAPI_KEY:
        print("Error: API key not found. Please check your keys.env file.")
        return  
    # Define the API endpoint for fetching top scorers (used here to check rate limits)
    url = "https://api-football-v1.p.rapidapi.com/v3/players/topscorers"
    # Set up the request headers, including the API key
    headers = {
        'X-RapidAPI-Key': RAPIDAPI_KEY,  
        'X-RapidAPI-Host': 'api-football-v1.p.rapidapi.com'  
    }
    try:
        # Send a GET request to the API
        response = requests.get(url, headers=headers)
        # Raise an exception if the response status code indicates an error (e.g., 401, 403, 429)
        response.raise_for_status()
        # Extract rate limit headers from the response
        rate_limits = {
            'daily_limit': response.headers.get('x-ratelimit-requests-limit'),  # Total requests allowed per day
            'daily_remaining': response.headers.get('x-ratelimit-requests-remaining'),  # Requests left for today
            'minute_limit': response.headers.get('x-ratelimit-limit'),  # Requests allowed per minute
            'minute_remaining': response.headers.get('x-ratelimit-remaining')  # Requests left in the current minute
        }
        # Print the extracted rate limits
        print(rate_limits)
    except requests.exceptions.RequestException as e:
        # Handle any request-related errors (e.g., network failure, API errors)
        print("Request failed:", e)
check_rate_limits()


{'daily_limit': '100', 'daily_remaining': '76', 'minute_limit': None, 'minute_remaining': None}


In [4]:
#exemple of fetching first top scorer of 24/25 (til now ) the french league 1  from the API 
# Load environment variables
load_dotenv('keys.env')

# Read API key
RAPIDAPI_KEY = os.getenv('RAPIDAPI_KEY')

# Check if the API key is loaded properly
if not RAPIDAPI_KEY:
    print("Error: API key not found. Please check your keys.env file.")
else:
    print("API Key Loaded Successfully")

# Set up API URL and parameters
url = "https://api-football-v1.p.rapidapi.com/v3/players/topscorers"
headers = {
    'X-RapidAPI-Key': RAPIDAPI_KEY,
    'X-RapidAPI-Host': 'api-football-v1.p.rapidapi.com'
}
params = {"league": "140", "season": "2024"}

# Make the API request
response = requests.get(url, headers=headers, params=params)

# Check status code before parsing JSON
print("Status Code:", response.status_code)

# If successful, print JSON response
if response.status_code == 200:
    data = response.json()
    # Extract information for the first player
    first_player_info = data['response'][0]
    # Display the first player's information in JSON format
    print(json.dumps(first_player_info, indent=4))
else:
    print("Error:", response.text)  # Print raw response for debugging


API Key Loaded Successfully
Status Code: 200
{
    "player": {
        "id": 521,
        "name": "R. Lewandowski",
        "firstname": "Robert",
        "lastname": "Lewandowski",
        "age": 37,
        "birth": {
            "date": "1988-08-21",
            "place": "Warszawa",
            "country": "Poland"
        },
        "nationality": "Poland",
        "height": "185 cm",
        "weight": "81 kg",
        "injured": false,
        "photo": "https://media.api-sports.io/football/players/521.png"
    },
    "statistics": [
        {
            "team": {
                "id": 529,
                "name": "Barcelona",
                "logo": "https://media.api-sports.io/football/teams/529.png"
            },
            "league": {
                "id": 140,
                "name": "La Liga",
                "country": "Spain",
                "logo": "https://media.api-sports.io/football/leagues/140.png",
                "flag": "https://media.api-sports.io/flags/es.svg",

In [6]:

# Load environment variables from .env file
load_dotenv('keys.env')

# Load API key to make API requests
RAPIDAPI_KEY = os.getenv('RAPIDAPI_KEY')

# Set up API request headers to authenticate requests
headers = {
    'X-RapidAPI-Key': RAPIDAPI_KEY,
    'X-RapidAPI-Host': 'api-football-v1.p.rapidapi.com'
}
# Set up API URL and parameters
url = "https://api-football-v1.p.rapidapi.com/v3/players/topscorers"
#select the french League 1 and 24/25 season
params = {"league":"140","season":"2024"}


In [7]:
#extract the data from The API and handle the errors
def get_top_scorers(url, headers, params):
    try:
        response = requests.get(url, headers=headers, params=params)
        response.raise_for_status()
        return response.json()
    except requests.exceptions.HTTPError as http_error_message:
        print (f"❌ [HTTP ERROR]: {http_error_message}")

    except requests.exceptions.ConnectionError as connection_error_message:
        print (f"❌ [CONNECTION ERROR]: {connection_error_message}")
    except requests.exceptions.Timeout as timeout_error_message:
        print (f"❌ [TIMEOUT ERROR]: {timeout_error_message}")
    except requests.exceptions.RequestException as other_error_message:
        print (f"❌ [UNKNOWN ERROR]: {other_error_message}")
get_top_scorers(url, headers, params)

{'get': 'players/topscorers',
 'parameters': {'league': '140', 'season': '2024'},
 'errors': [],
 'results': 20,
 'paging': {'current': 1, 'total': 1},
 'response': [{'player': {'id': 521,
    'name': 'R. Lewandowski',
    'firstname': 'Robert',
    'lastname': 'Lewandowski',
    'age': 37,
    'birth': {'date': '1988-08-21', 'place': 'Warszawa', 'country': 'Poland'},
    'nationality': 'Poland',
    'height': '185 cm',
    'weight': '81 kg',
    'injured': False,
    'photo': 'https://media.api-sports.io/football/players/521.png'},
   'statistics': [{'team': {'id': 529,
      'name': 'Barcelona',
      'logo': 'https://media.api-sports.io/football/teams/529.png'},
     'league': {'id': 140,
      'name': 'La Liga',
      'country': 'Spain',
      'logo': 'https://media.api-sports.io/football/leagues/140.png',
      'flag': 'https://media.api-sports.io/flags/es.svg',
      'season': 2024},
     'games': {'appearences': 25,
      'lineups': 24,
      'minutes': 2016,
      'number': Non

In [8]:
#This step represents the transformation phase of the ETL pipeline (T).
def process_top_scorers(data):
    """
    Parse the JSON data required for the top scorers 
    """
    top_scorers = []
    for scorer_data in data['response']:#The data we need is found in the response key. Response is an array of objects with all the detailed information we need about each top goal scorer.
        statistics = scorer_data['statistics'][0]

        # Set up constants for processing data 
        player = scorer_data['player']
        player_name = player['name']
        club_name = statistics['team']['name']
        club_logo = statistics['team']['logo']
        total_goals = int(statistics['goals']['total'])
        penalty_goals = int(statistics['penalty']['scored'])
        total_shots = int(statistics['shots']['total'])
        assists = int(statistics['goals']['assists']) if statistics['goals']['assists'] else 0
        matches_played = int(statistics['games']['appearences'])
        minutes_played = int(statistics['games']['minutes'])
        dob = datetime.strptime(player['birth']['date'], '%Y-%m-%d')
        age = (datetime.now() - dob).days // 365

        # Append data 
        top_scorers.append({
            'player': player_name,
            'club': club_name,
            'total_goals': total_goals,
            'penalty_goals': penalty_goals,
            'assists': assists,
            'matches': matches_played,
            'mins': minutes_played,
            'age': age,
            'total_shots': total_shots,
            'club_logo':club_logo
        })
    return top_scorers
process_top_scorers(get_top_scorers(url, headers, params))

[{'player': 'R. Lewandowski',
  'club': 'Barcelona',
  'total_goals': 21,
  'penalty_goals': 3,
  'assists': 2,
  'matches': 25,
  'mins': 2016,
  'age': 36,
  'total_shots': 67,
  'club_logo': 'https://media.api-sports.io/football/teams/529.png'},
 {'player': 'Kylian Mbappé',
  'club': 'Real Madrid',
  'total_goals': 18,
  'penalty_goals': 5,
  'assists': 3,
  'matches': 25,
  'mins': 2159,
  'age': 26,
  'total_shots': 83,
  'club_logo': 'https://media.api-sports.io/football/teams/541.png'},
 {'player': 'A. Budimir',
  'club': 'Osasuna',
  'total_goals': 14,
  'penalty_goals': 6,
  'assists': 2,
  'matches': 26,
  'mins': 1995,
  'age': 33,
  'total_shots': 50,
  'club_logo': 'https://media.api-sports.io/football/teams/727.png'},
 {'player': 'Raphinha',
  'club': 'Barcelona',
  'total_goals': 13,
  'penalty_goals': 1,
  'assists': 7,
  'matches': 26,
  'mins': 2135,
  'age': 28,
  'total_shots': 60,
  'club_logo': 'https://media.api-sports.io/football/teams/529.png'},
 {'player': 'Oi

In [9]:
def create_dataframe(top_scorers):
    """
    Convert list of dictionaries into a Pandas dataframe and process it
    """

    df = pd.DataFrame(top_scorers)

    # Sort dataframe first by 'total_goals' in descending order, then by 'assists' in descending order
    df.sort_values(by=['total_goals', 'assists'], ascending=[False, False], inplace=True)

    # Reset index after sorting to reflect new order
    df.reset_index(drop=True, inplace=True)

    # Recalculate ranks based on the sorted order of 'total_goals' using dense method 
    df['position'] = df['total_goals'].rank(method='dense', ascending=False).astype(int)

    # Specify the columns to include in the final dataframe in the desired order
    df = df[['position', 'player', 'club', 'total_goals', 'penalty_goals', 'assists', 'matches', 'mins', 'age','total_shots','club_logo']]

    return df
create_dataframe(process_top_scorers(get_top_scorers(url, headers, params)))

,position,player,club,total_goals,penalty_goals,assists,matches,mins,age,total_shots,club_logo
0,1,R. Lewandowski,Barcelona,21,3,2,25,2016,36,67,https://media.api-sports.io/football/teams/529...
1,2,Kylian Mbappé,Real Madrid,18,5,3,25,2159,26,83,https://media.api-sports.io/football/teams/541...
2,3,A. Budimir,Osasuna,14,6,2,26,1995,33,50,https://media.api-sports.io/football/teams/727...
3,4,Raphinha,Barcelona,13,1,7,26,2135,28,60,https://media.api-sports.io/football/teams/529...
4,4,Oihan Sancet,Athletic Club,13,2,1,21,1357,24,33,https://media.api-sports.io/football/teams/531...
5,5,D. Lukebakio,Sevilla,11,1,1,27,2261,27,55,https://media.api-sports.io/football/teams/536...
6,5,Ayoze Pérez,Villarreal,11,0,0,19,1168,31,35,https://media.api-sports.io/football/teams/533...
7,5,Kike García,Alaves,11,2,0,26,1782,35,31,https://media.api-sports.io/football/teams/542...
8,6,Vinícius Júnior,Real Madrid,10,2,5,21,1721,24,47,https://media.api-sports.io/football/teams/541...
9,6,J. Álvarez,Atletico Madrid,10,1,2,27,1766,25,41,https://media.api-sports.io/football/teams/530...


In [7]:
load_dotenv('keys.env')
# Load MySQL database credentials
SERVER = os.getenv('SERVER')
MY_DATABASE = os.getenv('MY_DATABASE')

In [8]:

def create_db_connection(server_name, db_name):
    """
    Establish a connection to the SQL Server database using Windows Authentication.
    """
    db_connection = None
    try:
        db_connection = pyodbc.connect(
            f'DRIVER={{SQL Server}};'
            f'SERVER={server_name};'
            f'DATABASE={db_name};'
            f'Trusted_Connection=yes;'
        )
        print("✅ SQL Server Database connection successful!")

    except pyodbc.Error as e:
        print(f"❌ [DATABASE CONNECTION ERROR]: {e}")

    return db_connection

create_db_connection(SERVER, MY_DATABASE)

✅ SQL Server Database connection successful!


In [9]:
def create_table(db_connection):
    """
    Create a table if it does not exist in the SQL Server database
    """
    CREATE_TABLE_SQL_QUERY = """
    IF NOT EXISTS (SELECT * FROM sysobjects WHERE name='top_scorers' AND xtype='U')
    CREATE TABLE top_scorers (
        position INT,
        player VARCHAR(255),
        club VARCHAR(255),
        total_goals INT,
        penalty_goals INT,
        assists INT,
        matches INT,
        mins INT,
        age INT,
        total_shots INT, 
        club_logo VARCHAR(255),
        PRIMARY KEY (player, club)
    );
    """
    
    try:
        cursor = db_connection.cursor()
        cursor.execute(CREATE_TABLE_SQL_QUERY)
        db_connection.commit()
        print("✅ Table 'top_scorers' created successfully!")

    except pyodbc.Error as e:
        print(f"❌ [CREATING TABLE ERROR]: {e}")
create_table(create_db_connection(SERVER, MY_DATABASE))

✅ SQL Server Database connection successful!
✅ Table 'top_scorers' created successfully!


In [10]:
# Insert data into the table
def insert_into_table(db_connection, df):
    """
    Insert or update the top scorers data in the database from the dataframe
    """
    cursor = db_connection.cursor()

    INSERT_DATA_SQL_QUERY = """
    MERGE INTO top_scorers AS target
    USING (VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)) AS source (position, player, club, total_goals, penalty_goals, assists, matches, mins, age, total_shots, club_logo)
    ON target.player = source.player AND target.club = source.club
    WHEN MATCHED THEN
        UPDATE SET
            target.total_goals = source.total_goals,
            target.penalty_goals = source.penalty_goals,
            target.assists = source.assists,
            target.matches = source.matches,
            target.mins = source.mins,
            target.age = source.age,
            target.total_shots = source.total_shots,  -- Update this line if the column name is different
            target.club_logo = source.club_logo
    WHEN NOT MATCHED THEN
        INSERT (position, player, club, total_goals, penalty_goals, assists, matches, mins, age, total_shots, club_logo)  -- Update this line if the column name is different
        VALUES (source.position, source.player, source.club, source.total_goals, source.penalty_goals, source.assists, source.matches, source.mins, source.age, source.total_shots, source.club_logo);
    """

    # Create a list of tuples from the dataframe values
    data_values_as_tuples = [tuple(x) for x in df.to_numpy()]

    # Execute the query
    cursor.executemany(INSERT_DATA_SQL_QUERY, data_values_as_tuples)
    db_connection.commit()
    print("Data inserted or updated successfully ✅")
insert_into_table(create_db_connection(SERVER, MY_DATABASE), create_dataframe(process_top_scorers(get_top_scorers(url, headers, params))))


✅ SQL Server Database connection successful!
Data inserted or updated successfully ✅


In [5]:
def run_data_pipeline():
    """
    Execute the ETL pipeline 
    """
    check_rate_limits()

    data = get_top_scorers(url, headers, params)

    if data and 'response' in data and data['response']:
        top_scorers = process_top_scorers(data)
        df = create_dataframe(top_scorers)
        print(df.to_string(index=False)) 

    else:
        print("No data available or an error occurred ❌")

    db_connection = create_db_connection(SERVER, MY_DATABASE)


    # If connection is successful, proceed with creating table and inserting data
    if db_connection is not None:
        create_table(db_connection)  
        df = create_dataframe(top_scorers) 
        insert_into_table(db_connection, df)  

if __name__ == "__main__":
    run_data_pipeline()


{'daily_limit': '100', 'daily_remaining': '75', 'minute_limit': None, 'minute_remaining': None}


NameError: name 'get_top_scorers' is not defined